In [ ]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, BasicAer
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
"""Circuit for adding two integers, The circuit works on two registers holding the input integers,
the first of size n qubits (x), the second of size n qubits (y), plus a carry qubit (c).
It operates in place, i.e. the contents of the second register are replaced to hold the sum of the inputs
storing a possible carry bit in the additionally available qubit."""
def integer_addition_Takahashi(circuit,x,y,c,N):
    for i in range(1,N):
        circuit.cx(x[i],y[i])
    
    circuit.cx(x[N-1],c)
    
    for i in range(N-2,0,-1):
        circuit.cx(x[i],x[i+1])
    
    for i in range(0,N-1):
        circuit.ccx(y[i],x[i],x[i+1])
    
    circuit.ccx(y[N-1],x[N-1],c)
    
    for i in range(N-1,0,-1):
        circuit.cx(x[i],y[i])
        circuit.ccx(y[i-1],x[i-1],x[i])
    
    for i in range(1,N-1):
        circuit.cx(x[i],x[i+1])
    
    for i in range(0,N):
        circuit.cx(x[i],y[i])

In [ ]:
def integer_addition_Takahashi_inverse(circuit,x,y,c,N):
    for i in range(N-1,-1,-1):
        circuit.cx(x[i],y[i])
    
    for i in range(N-2,0,-1):
        circuit.cx(x[i],x[i+1])
    
    for i in range(1,N):
        circuit.ccx(y[i-1],x[i-1],x[i])
        circuit.cx(x[i],y[i])
    
    circuit.ccx(y[N-1],x[N-1],c)
    
    for i in range(N-2,-1,-1):
        circuit.ccx(y[i],x[i],x[i+1])
    
    for i in range(1,N-1):
        circuit.cx(x[i],x[i+1])
    
    circuit.cx(x[N-1],c)
    
    for i in range(N-1,0,-1):
        circuit.cx(x[i],y[i])

In [ ]:
def test_add(a,b,n):
    x = QuantumRegister(n)
    y = QuantumRegister(n)
    c = QuantumRegister(1)
    clas = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    circuit = QuantumCircuit(x,y,c,clas,clas_c)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])
    integer_addition_Takahashi(circuit,x,y,c,n)
    for i in range(0, n):
        circuit.measure(y[i],clas[i])
    circuit.measure(c,clas_c)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_b = output[1]
    output_c = output[0]
    res_b = int(output_b, 2)
    res_c = int(output_c, 2)
    print(a,b,n)
    print(res_b,res_c)
    if( res_b!=(b+a)%2**n or res_c!=((a+b)>=2**n) ):
        print('found!')

In [ ]:
def test_sub(a,b,n):
    x = QuantumRegister(n)
    y = QuantumRegister(n)
    c = QuantumRegister(1)
    clas = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    circuit = QuantumCircuit(x,y,c,clas,clas_c)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])
    integer_addition_Takahashi_inverse(circuit,x,y,c,n)
    for i in range(0, n):
        circuit.measure(y[i],clas[i])
    circuit.measure(c,clas_c)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_b = output[1]
    output_c = output[0]
    res_b = int(output_b, 2)
    res_c = int(output_c, 2)
    print(a,b,n)
    print(res_b,res_c)
    if( res_b!=(b-a)%2**n or res_c!=(a>b) ):
        print('found!')

In [ ]:
for i in range (0,2**5,3):
    for j in range (0,2**5,7):
        test_add(i,j,5)
        print('-----------')

In [ ]:
for i in range (0,2**5,3):
    for j in range (0,2**5,7):
        test_sub(i,j,5)
        print('-----------')